1.При выборе модели мы обычно выбираем ту, которая дает наибольшее значение какой-то метрики. Но это означает, что эта оценка метрики уже становится завышенной. Поэтому для объективной оценки качества модели ее опять следует оценить на новом наборе данных. Поэтому для выбора модели нужно разделить выборку на три части - обучающую, валидационную и тестовую. Повторите первую часть работы, но с таким разбиением и получите несмещенную оценку качества наилучшей модели.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
X, Y = make_classification(n_samples=1000, n_classes=2, n_features=5, n_redundant=0, random_state=1)

In [ ]:
# Разбиваем данные на обучающую, валидационную и тестовую выборки
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.3, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [ ]:
names = ["Nearest_Neighbors", "Linear_SVM", "Polynomial_SVM", "RBF_SVM",
        "Decision_Tree", "Random_Forest", "Neural_Net", "Naive_Bayes"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(kernel="poly", degree=3, C=0.025),
    SVC(kernel="rbf", C=1, gamma=2),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=100),
    MLPClassifier(alpha=1, max_iter=1000),
    GaussianNB()]

In [ ]:
scores_train = []
scores_test = []
scores_val = []
for name, clf in zip(names, classifiers):
    clf.fit(X_train, Y_train)
    y_pred_train = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)
    y_pred_val = clf.predict(X_val)
    acc_train = accuracy_score(Y_train, y_pred_train)
    acc_test = accuracy_score(Y_test, y_pred_test)
    acc_val = accuracy_score(Y_val, y_pred_val)

    scores_train.append(acc_train)
    scores_test.append(acc_test)
    scores_val.append(acc_val)

In [ ]:
df = pd.DataFrame()
df['name'] = names
df['accuracy_train'] = scores_train
df['accuracy_test'] = scores_test
df['accuracy_val'] = scores_val
df.sort_values(by=["accuracy_test"], ascending=False, inplace=True)
df

,name,accuracy_train,accuracy_test,accuracy_val
0,Nearest_Neighbors,0.888571,0.880000,0.826667
6,Neural_Net,0.880000,0.880000,0.860000
4,Decision_Tree,0.908571,0.873333,0.846667
1,Linear_SVM,0.855714,0.866667,0.846667
5,Random_Forest,0.895714,0.866667,0.853333
3,RBF_SVM,0.977143,0.860000,0.846667
7,Naive_Bayes,0.855714,0.860000,0.853333
2,Polynomial_SVM,0.795714,0.800000,0.766667


Наилучший результат на тестовой выборке был получен при использовании модели К-ближайших соседий.

2. Оптимизируйте гиперпараметры той же модели, но другим методом - случайным поиском. Сравните полученные результаты.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    'n_neighbors': [1, 10, 66, 100, 300]
}

k_neighbors = KNeighborsClassifier()
grid_search = GridSearchCV(estimator=k_neighbors,
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=2,
                           n_jobs=-1)

In [ ]:
grid_search.fit(X_train, Y_train)
best_params = grid_search.best_params_

best_model = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'])
best_model.fit(X_train, Y_train)

y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)
y_pred_val = clf.predict(X_val)
acc_train = accuracy_score(Y_train, y_pred_train)
acc_test = accuracy_score(Y_test, y_pred_test)
acc_val = accuracy_score(Y_val, y_pred_val)

print(f"Лучшие параметры: {best_params}")
print(f"Точность на обучающем наборе: {acc_train.round(3)}")
print(f"Точность на тестовом наборе: {acc_test.round(3)}")
print(f"Точность на валидационном наборе: {acc_val.round(3)}")

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Лучшие параметры: {'n_neighbors': 66}
Точность на обучающем наборе: 0.856
Точность на тестовом наборе: 0.86
Точность на валидационном наборе: 0.853


3. Во второй задаче найдите наиболее эффективный класс моделей, а затем оптимизируйте гиперпараметры у этой модели. Сравните, насколько лучше получилась модель.

In [ ]:
scores_val = []
scores_train = []
scores_test = []
for name, clf in zip(names, classifiers):
    clf.fit(X_train, Y_train)
    y_pred_train = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)
    y_pred_val = clf.predict(X_val)
    acc_train = accuracy_score(Y_train, y_pred_train)
    acc_test = accuracy_score(Y_test, y_pred_test)
    acc_val = accuracy_score(Y_val, y_pred_val)

    scores_train.append(acc_train)
    scores_test.append(acc_test)
    scores_val.append(acc_val)

df_val = pd.DataFrame()
df_val['name'] = names
df_val['accuracy_val'] = scores_val
df_val['accuracy_train'] = scores_train
df_val['accuracy_test'] = scores_test
df_val.sort_values(by=["accuracy_val"], ascending=False, inplace=True)
df_val

,name,accuracy_val,accuracy_train,accuracy_test
5,Random_Forest,0.853333,0.902857,0.873333
6,Neural_Net,0.853333,0.882857,0.880000
7,Naive_Bayes,0.853333,0.855714,0.860000
1,Linear_SVM,0.846667,0.855714,0.866667
3,RBF_SVM,0.846667,0.977143,0.860000
4,Decision_Tree,0.846667,0.908571,0.873333
0,Nearest_Neighbors,0.826667,0.888571,0.880000
2,Polynomial_SVM,0.766667,0.795714,0.800000


In [ ]:
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (150,), (200,)],
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1]
}

mlp_clf = MLPClassifier()
grid_search = GridSearchCV(estimator=mlp_clf,
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=2,
                           n_jobs=-1)

grid_search.fit(X_train, Y_train)
best_params = grid_search.best_params_

best_model = grid_search.best_estimator_

y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)
y_pred_val = best_model.predict(X_val)
acc_train = accuracy_score(Y_train, y_pred_train)
acc_test = accuracy_score(Y_test, y_pred_test)
acc_val = accuracy_score(Y_val, y_pred_val)

print(f"Лучшие параметры: {best_params}")
print(f"Точность на обучающем наборе: {acc_train.round(3)}")
print(f"Точность на тестовом наборе: {acc_test.round(3)}")
print(f"Точность на валидационном наборе: {acc_val.round(3)}")

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Лучшие параметры: {'alpha': 0.001, 'hidden_layer_sizes': (100,)}
Точность на обучающем наборе: 0.881
Точность на тестовом наборе: 0.88
Точность на валидационном наборе: 0.853


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
